# --- Day 7: Internet Protocol Version 7 ---

While snooping around the local network of EBHQ, you compile a list of IP addresses (they're IPv7, of course; IPv6 is much too limited). You'd like to figure out which IPs support TLS (transport-layer snooping).

An IP supports TLS if it has an Autonomous Bridge Bypass Annotation, or ABBA. An ABBA is any four-character sequence which consists of a pair of two different characters followed by the reverse of that pair, such as xyyx or abba. However, the IP also must not have an ABBA within any hypernet sequences, which are contained by square brackets.

For example:

- `abba[mnop]qrst` supports TLS (abba outside square brackets).
- `abcd[bddb]xyyx` does not support TLS (bddb is within square brackets, even though xyyx is outside square brackets).
- `aaaa[qwer]tyui` does not support TLS (aaaa is invalid; the interior characters must be different).
- `ioxxoj[asdfgh]zxcvbn` supports TLS (oxxo is outside square brackets, even though it's within a larger string).

**How many IPs in your puzzle input support TLS?**

In [52]:
# the puzzle input
with open('inputs/7.txt') as f:
    data = f.read().strip().split("\n")
data[:3]

['rhamaeovmbheijj[hkwbkqzlcscwjkyjulk]ajsxfuemamuqcjccbc',
 'gdlrknrmexvaypu[crqappbbcaplkkzb]vhvkjyadjsryysvj[nbvypeadikilcwg]jwxlimrgakadpxu[dgoanojvdvwfabtt]yqsalmulblolkgsheo',
 'dqpthtgufgzjojuvzvm[eejdhpcqyiydwod]iingwezvcbtowwzc[uzlxaqenhgsebqskn]wcucfmnlarrvdceuxqc[dkwcsxeitcobaylhbvc]klxammurpqgmpsxsr']

First up a function to detect if four chars are ABBA:

In [64]:
def abba(a):
    """takes in a string and returns true if ABBA"""
    return a[:2] == a[2:][::-1] and a[0] != a[1]

abba("abba")

True

In [60]:
def is_tls(ip):
    """takes in a ip address and returns True if TLS"""
    tls = False
    
    in_brackets = False
    
    for i in range(len(ip)-3):
        if ip[i] == "[": in_brackets = True
        if ip[i] == "]": in_brackets = False
        
        if abba(ip[i:i+4]) and not in_brackets:
            tls = True
        if in_brackets and abba(ip[i:i+4]):
            return False # if ABBA found inside brackets no need to continue
    
    return tls

is_tls("abba[mnop]qrst"), is_tls("abcd[bddb]xyyx"), is_tls("ioxxoj[asdfgh]zxcvbn")

(True, False, True)

In [63]:
sum([is_tls(ip) for ip in data])

110

# --- Part Two ---

You would also like to know which IPs support SSL (super-secret listening).

An IP supports SSL if it has an Area-Broadcast Accessor, or ABA, anywhere in the supernet sequences (outside any square bracketed sections), and a corresponding Byte Allocation Block, or BAB, anywhere in the hypernet sequences. An ABA is any three-character sequence which consists of the same character twice with a different character between them, such as xyx or aba. A corresponding BAB is the same characters but in reversed positions: yxy and bab, respectively.

For example:

- aba[bab]xyz supports SSL (aba outside square brackets with corresponding bab within square brackets).
- xyx[xyx]xyx does not support SSL (xyx, but no corresponding yxy).
- aaa[kek]eke supports SSL (eke in supernet with corresponding kek in hypernet; the aaa sequence is not related, because the interior character must be different).
- zazbz[bzb]cdb supports SSL (zaz has no corresponding aza, but zbz has a corresponding bzb, even though zaz and zbz overlap).

**How many IPs in your puzzle input support SSL?**

In [120]:
def aba(a):
    """takes in a string and returns true if in form ABA"""
    assert len(a) == 3
    return a[0] == a[2] and a[0] != a[1]

aba("aba")

True

In [121]:
def is_ssl(ip):
    """takes in a ip address and returns True if ssl"""
    
    abas, babs = [], []
    
    in_brackets = False
    
    for i in range(len(ip)-2):
        if ip[i] == "[": in_brackets = True
        if ip[i] == "]": in_brackets = False
        
        seq = ip[i:i+3]
        if "[" not in seq and "]" not in seq:
            if aba(seq) and not in_brackets:
                abas.append(seq)

            if in_brackets and aba(seq):
                babs.append(seq) 
    
    # reversing the aba's into bab's to make it easy to compare
    abas_r = [i[1]+i[0]+i[1] for i in abas]
    
    if set(abas_r) & set(babs):
        return True
    else:
        return False

is_ssl("aba[bab]xyz"), is_ssl("xyx[xyx]xyx")

(True, False)

In [122]:
sum([is_ssl(ip) for ip in data])

242

# Notes

- sets! use sets!
- use regex